In [46]:
import requests
from bs4 import BeautifulSoup
import json

def get_reviews(product_id, page=1):
    url = f"https://tiki.vn/api/v2/reviews?product_id={product_id}&limit=10&page={page}&include=comments"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36",
        "Accept": "application/json",
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data = response.json()
        reviews = data.get('data', [])
        return reviews
    else:
        print(f"Failed to fetch reviews: {response.status_code}")
        return []

def save_reviews_to_file(reviews, filename="reviews.json"):
    with open(filename, "w", encoding="utf-8") as f:
        json.dump(reviews, f, ensure_ascii=False, indent=4)

def remove_duplicates(reviews):
    # Dùng set để loại bỏ trùng lặp dựa trên nội dung bình luận
    seen = set()
    unique_reviews = []
    for review in reviews:
        content = review.get('content', '').strip()
        if content not in seen:
            seen.add(content)
            unique_reviews.append(review)
    return unique_reviews


def main():
    product_id = 3480333  # ID của sản phẩm từ URL
    all_reviews = []
    for page in range(1, 10):  # Crawl 9 trang
        print(f"Fetching page {page}...")
        reviews = get_reviews(product_id, page)
        if not reviews:
            break
        all_reviews.extend(reviews)
    
    print(f"Total reviews fetched: {len(all_reviews)}")

    # Loại bỏ bình luận trùng lặp
    unique_reviews = remove_duplicates(all_reviews)
    print(f"Total unique reviews: {len(unique_reviews)}")

    # Lưu các bình luận đã loại bỏ trùng lặp
    save_reviews_to_file(unique_reviews)

if __name__ == "__main__":
    main()


Fetching page 1...
Fetching page 2...
Fetching page 3...
Fetching page 4...
Fetching page 5...
Fetching page 6...
Fetching page 7...
Fetching page 8...
Fetching page 9...
Total reviews fetched: 90
Total unique reviews: 87


In [47]:
import json
import csv

def extract_reviews_to_csv(json_file, csv_file):
    with open(json_file, "r", encoding="utf-8") as f:
        data = json.load(f)
    
    filtered_reviews = []
    
    # Lọc bình luận và nhãn
    for review in data:
        content = review.get("content", "").strip()
        
        if content:
            filtered_reviews.append({"content": content})
    
    # Ghi dữ liệu vào file CSV
    with open(csv_file, "w", encoding="utf-8-sig", newline="") as f:
        writer = csv.DictWriter(f, fieldnames=["content"])
        writer.writeheader()
        writer.writerows(filtered_reviews)
    
    print(f"Extracted {len(filtered_reviews)} reviews and saved to {csv_file}")

# Sử dụng
input_file = "reviews.json"  # Tệp JSON đầu vào
output_csv = "reviews.csv"  # Tệp CSV đầu ra
extract_reviews_to_csv(input_file, output_csv)


Extracted 86 reviews and saved to reviews.csv


In [48]:
df = pd.read_csv("reviews.csv")
print(df.head())

                                             content
0  Shop mới vừa chạy chương trình tặng ly nhựa 7 ...
1  Đã nhận hàng như quảng cáo, hàng đẹp, chất lượ...
2  Tiki là một thế giới rất diệu kỳ dành cho nhữn...
3  Hàng chưa sử dụng nên chưa biết thế nào. Nhưng...
4  Ấm đẹp, lòng ấm inox liền khối, sôi nhanh max ...


In [50]:
# Đọc tệp stopword_vietnamese.txt và tạo danh sách từ dừng
def load_stopwords(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        stopwords = file.readlines()
    stopwords = [word.strip() for word in stopwords]  # Loại bỏ khoảng trắng thừa
    return stopwords

# Tải stopwords
stop_words = load_stopwords("stopwords_vietnamese.txt")

# Loại bỏ từ dừng
def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in stop_words])

# Chuấn hóa dữ liệu
def standardize_data(row):
    # Convert to lowercase
    row = row.lower()
    
    # Remove stopwords (if you have a list of stopwords, you can use it here)
    row = row.replace(",", " ").replace(".", " ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ") \
        .replace("(", " ").replace(")", " ")

    row = row.strip()
    return row

df['content'] = df['content'].apply(standardize_data).apply(remove_stopwords)
print(df.head())

# Lưu dữ liệu đã xử lý vào file mới
output_file = "processed_reviews.csv"
df.to_csv(output_file, index=False, encoding="utf-8-sig")


                                             content
0  shop chạy chương trình tặng ly nhựa 7 tây 8 tâ...
1  hàng quảng cáo hàng đẹp chất thời gian sử dụng...
2  tiki giới diệu kỳ đam mê mua sắm mua giá rẻ fr...
3  hàng sử dụng giao hàng văn hóa tan tầm đt chạy...
4  ấm đẹp ấm inox liền khối sôi max ấm 4 phút nhi...
